<a href="https://colab.research.google.com/github/mydreamisto/notebooks/blob/main/course/en/chapter3/section3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning a model with the Trainer API or Keras

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets

In [ ]:
!pip install wandb

In [ ]:
!pip install evaluate

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Training

In [ ]:
# TrainingArguments class contains all the hyperparameters the Trainer will use for training and evaluation
from transformers import TrainingArguments
# The only argument you have to provide is a directory where the trained model will be saved, as well as the checkpoints along the way
training_args = TrainingArguments("test-trainer")

In [ ]:
from transformers import AutoModelForSequenceClassification
# num_labels=2 表示该序列分类任务是一个二分类任务
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Trainer class to help you fine-tune any of the pretrained models it provides on your dataset.
from transformers import Trainer
trainer = Trainer(
    model,
    training_args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["validation"],
    data_collator = data_collator
)

In [ ]:
import wandb
"""
导入了 wandb 库，这是一个用于实验跟踪和可视化的工具。
wandb.init() 是 wandb 库的初始化函数，它会创建一个新的实验运行，设置实验的一些初始信息，如实验名称、超参数等。
如果不调用这个函数，当后续代码中调用 wandb 的其他函数（如 wandb.log()）时，程序将无法正常工作，因为它不知道将数据记录到哪个实验中。
然后，调用 trainer.train() 进行训练。在训练过程中，程序可能会使用 wandb 来记录训练的各种信息，例如损失、指标等，通过调用 wandb.log() 来实现。
由于我们已经调用了 wandb.init()，所以 wandb.log() 可以正常工作，将数据发送到 wandb 的服务器进行存储和可视化。
"""
wandb.init()

# it would reports the training loss every 500 steps
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,0.206700
1000,0.123600


TrainOutput(global_step=1377, training_loss=0.1310880476786074, metrics={'train_runtime': 208.0578, 'train_samples_per_second': 52.889, 'train_steps_per_second': 6.618, 'total_flos': 405114969714960.0, 'train_loss': 0.1310880476786074, 'epoch': 3.0})

In [ ]:
# the output of predict() is a named tuple with 3 field:
# "predictions", "label_ids", "metrics"
# The "metrics" field will just contain the loss on the dataset passed,
# as well as some time metrics (how long it took to predict, in total and on average).

# 存储预测的结果
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

# As you can see, predictions is a two-dimensional array with shape 408 x 2
# (408 being the number of elements in the dataset we used).
# 2 是由之前提到的 num_labels 决定的（可能是一个二分类任务）
# Those are the logits for each element of the dataset we passed to predict()



(408, 2) (408,)


predictions 是一个二维数组。

对于每个样本，模型会输出一组 logits，在二分类任务中，这组 logits 包含两个元素，在多分类任务中会有更多元素，每个元素代表模型对该样本属于相应类别的 “得分”，但这些得分并不是概率。

logits 是模型的原始输出，在分类任务中，它们是未经 softmax 或 sigmoid 激活函数处理的值。对于二分类任务，logits 可以被解释为模型对每个类别的未归一化的 “得分”。例如，对于一个样本，logits 为 [2.3, -1.5] 表示模型更倾向于将该样本预测为第一类，因为第一类的得分更高。

predictions.predictions 的形状为 408 x 2（在这个例子中），其中 408 是传递给 predict() 方法的数据集的元素数量，2 是由之前提到的 num_labels 决定的（可能是一个二分类任务）。对于每个样本，predictions.predictions 存储了预测的结果，但这里存储的是 logits 而不是直接的类别标签。

形状为 (408,) 表示 predictions.label_ids 是一个一维数组，其中包含了 408 个元素，每个元素是一个整数，表示该样本的真实类别标签。对于第 i 个样本，其真实类别标签可以通过 predictions.label_ids[i] 获取。

In [ ]:
# To transform them into predictions that we can compare to our labels, we need to take the index with the maximum value on the second axis:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8602941176470589, 'f1': 0.9015544041450777}

In [ ]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-18-415be425bb02>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.434373,0.803922,0.859649
2,0.572700,0.412914,0.848039,0.891608
3,0.375600,0.676060,0.835784,0.886633


TrainOutput(global_step=1377, training_loss=0.41641840384157136, metrics={'train_runtime': 226.2499, 'train_samples_per_second': 48.636, 'train_steps_per_second': 6.086, 'total_flos': 405114969714960.0, 'train_loss': 0.41641840384157136, 'epoch': 3.0})